In [1]:
import pandas as pd
import numpy as np
#import kaleido
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import holidays
from datetime import date
from dateutil.relativedelta import relativedelta 
# This code below is to import an excel or csv for python to plot
import os


# Label Peaks

In [2]:
def clean_data(df, date_name, melt_vector = [], row  = True):
    if row:
        df = df.melt(id_vars= melt_vector, var_name='Time') 

    # Combine     
    df[date_name] = pd.to_datetime(df.Date.astype(str)+' '+df.Time.astype(str))
    # Delete time column since it is not needed
    df = df.drop(columns=['Time'])
    # Now python will sort the dataframe by time AND date
    df = df.sort_values([date_name])
    # This will renumber the resorted values from 0 to len(df)
    df = df.reset_index(drop=True)
    # This will rename the column  
    df = df.rename(columns={'value': 'Demand'})
    # This will convert everything to numeric
    df.Demand = pd.to_numeric(df.Demand, errors='coerce')
    
    return df

In [3]:
 # -*- coding: utf-8 -*-

#  python-holidays
#  ---------------
#  A fast, efficient Python library for generating country, province and state
#  specific sets of holidays on the fly. It aims to make determining whether a
#  specific date is a holiday as fast and flexible as possible.
#
#  Author:  ryanss <ryanssdev@icloud.com> (c) 2014-2017
#           dr-prodigy <maurizio.montel@gmail.com> (c) 2017-2020
#  Website: https://github.com/dr-prodigy/python-holidays
#  License: MIT (see LICENSE file)

from datetime import date, datetime, timedelta
from dateutil.easter import easter, EASTER_ORTHODOX
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta as rd
from dateutil.relativedelta import MO, TU, WE, TH, FR, SA, SU
import inspect
import six
import sys
import warnings

__version__ = '0.9.12'

MON, TUE, WED, THU, FRI, SAT, SUN = range(7)
WEEKEND = (SAT, SUN)

JAN, FEB, MAR, APR, MAY, JUN, JUL, AUG, SEP, OCT, \
    NOV, DEC = range(1, 13)


class HolidayBase(dict):
    PROVINCES = []

    def __init__(self, years=[], expand=True, observed=True,
                 prov=None, state=None):
        self.observed = observed
        self.expand = expand
        if isinstance(years, int):
            years = [years, ]
        self.years = set(years)
        if not getattr(self, 'prov', False):
            self.prov = prov
        self.state = state
        for year in list(self.years):
            self._populate(year)

    def __setattr__(self, key, value):
        if key == 'observed' and len(self) > 0:
            dict.__setattr__(self, key, value)
            if value is True:
                # Add (Observed) dates
                years = list(self.years)
                self.years = set()
                self.clear()
                for year in years:
                    self._populate(year)
            else:
                # Remove (Observed) dates
                for k, v in list(self.items()):
                    if v.find("Observed") >= 0:
                        del self[k]
        else:
            return dict.__setattr__(self, key, value)

    def __keytransform__(self, key):
        if isinstance(key, datetime):
            key = key.date()
        elif isinstance(key, date):
            key = key
        elif isinstance(key, int) or isinstance(key, float):
            key = datetime.utcfromtimestamp(key).date()
        elif isinstance(key, six.string_types):
            try:
                key = parse(key).date()
            except (ValueError, OverflowError):
                raise ValueError("Cannot parse date from string '%s'" % key)
        else:
            raise TypeError("Cannot convert type '%s' to date." % type(key))

        if self.expand and key.year not in self.years:
            self.years.add(key.year)
            self._populate(key.year)
        return key

    def __contains__(self, key):
        return dict.__contains__(self, self.__keytransform__(key))

    def __getitem__(self, key):
        if isinstance(key, slice):
            if not key.start or not key.stop:
                raise ValueError("Both start and stop must be given.")

            start = self.__keytransform__(key.start)
            stop = self.__keytransform__(key.stop)

            if key.step is None:
                step = 1
            elif isinstance(key.step, timedelta):
                step = key.step.days
            elif isinstance(key.step, int):
                step = key.step
            else:
                raise TypeError(
                    "Cannot convert type '%s' to int." % type(key.step)
                )

            if step == 0:
                raise ValueError('Step value must not be zero.')

            date_diff = stop - start
            if date_diff.days < 0 <= step or date_diff.days >= 0 > step:
                step *= -1

            days_in_range = []
            for delta_days in range(0, date_diff.days, step):
                day = start + timedelta(days=delta_days)
                try:
                    dict.__getitem__(
                        self,
                        day
                    )
                    days_in_range.append(day)
                except (KeyError):
                    pass
            return days_in_range
        return dict.__getitem__(self, self.__keytransform__(key))

    def __setitem__(self, key, value):
        if key in self:
            if self.get(key).find(value) < 0 \
                    and value.find(self.get(key)) < 0:
                value = "%s, %s" % (value, self.get(key))
            else:
                value = self.get(key)
        return dict.__setitem__(self, self.__keytransform__(key), value)

    def update(self, *args):
        args = list(args)
        for arg in args:
            if isinstance(arg, dict):
                for key, value in list(arg.items()):
                    self[key] = value
            elif isinstance(arg, list):
                for item in arg:
                    self[item] = "Holiday"
            else:
                self[arg] = "Holiday"

    def append(self, *args):
        return self.update(*args)

    def get(self, key, default=None):
        return dict.get(self, self.__keytransform__(key), default)

    def get_list(self, key):
        return [h for h in self.get(key, "").split(", ") if h]

    def pop(self, key, default=None):
        if default is None:
            return dict.pop(self, self.__keytransform__(key))
        return dict.pop(self, self.__keytransform__(key), default)

    def __eq__(self, other):
        return dict.__eq__(self, other) and self.__dict__ == other.__dict__

    def __ne__(self, other):
        return dict.__ne__(self, other) or self.__dict__ != other.__dict__

    def __add__(self, other):
        if isinstance(other, int) and other == 0:
            # Required to sum() list of holidays
            # sum([h1, h2]) is equivalent to (0 + h1 + h2)
            return self
        elif not isinstance(other, HolidayBase):
            raise TypeError()
        HolidaySum = createHolidaySum(self, other)
        country = (getattr(self, 'country', None) or
                   getattr(other, 'country', None))
        if self.country and other.country and self.country != other.country:
            c1 = self.country
            if not isinstance(c1, list):
                c1 = [c1]
            c2 = other.country
            if not isinstance(c2, list):
                c2 = [c2]
            country = c1 + c2
        prov = getattr(self, 'prov', None) or getattr(other, 'prov', None)
        if self.prov and other.prov and self.prov != other.prov:
            p1 = self.prov if isinstance(self.prov, list) else [self.prov]
            p2 = other.prov if isinstance(other.prov, list) else [other.prov]
            prov = p1 + p2
        return HolidaySum(years=(self.years | other.years),
                          expand=(self.expand or other.expand),
                          observed=(self.observed or other.observed),
                          country=country, prov=prov)

    def __radd__(self, other):
        return self.__add__(other)

    def _populate(self, year):
        pass


def createHolidaySum(h1, h2):
    class HolidaySum(HolidayBase):

        def __init__(self, country, **kwargs):
            self.country = country
            self.holidays = []
            if getattr(h1, 'holidays', False):
                for h in h1.holidays:
                    self.holidays.append(h)
            else:
                self.holidays.append(h1)
            if getattr(h2, 'holidays', False):
                for h in h2.holidays:
                    self.holidays.append(h)
            else:
                self.holidays.append(h2)
            HolidayBase.__init__(self, **kwargs)

        def _populate(self, year):
            for h in self.holidays[::-1]:
                h._populate(year)
                self.update(h)

    return HolidaySum


def list_supported_countries():
    """List all supported countries incl. their abbreviation."""
    return [name for name, obj in
            inspect.getmembers(sys.modules[__name__], inspect.isclass)
            if obj.__module__ is __name__]


def CountryHoliday(country, years=[], prov=None, state=None, expand=True,
                   observed=True):
    try:
        country_holiday = globals()[country](years=years,
                                             prov=prov,
                                             state=state,
                                             expand=expand,
                                             observed=observed)
    except (KeyError):
        raise KeyError("Country %s not available" % country)
    return country_holiday

 class UsElectricHolidays(holidays.HolidayBase):
 
    def _populate(self, year):
         # New Year's Day
        if year > 1870:
            name = "New Year's Day"
            self[date(year, JAN, 1)] = name
            if self.observed and date(year, JAN, 1).weekday() == SUN:
                self[date(year, JAN, 1) + rd(days=+1)] = name + \
                    " (Observed)"

        # Washington's Birthday
        name = "Washington's Birthday"
        if year > 1970:
            self[date(year, FEB, 1) + rd(weekday=MO(+3))] = name
        elif year >= 1879:
            self[date(year, FEB, 22)] = name

        # Memorial Day
        if year > 1970:
            self[date(year, MAY, 31) + rd(weekday=MO(-1))] = "Memorial Day"
        elif year >= 1888:
            self[date(year, MAY, 30)] = "Memorial Day"

        # Independence Day
        if year > 1870:
            name = "Independence Day"
            self[date(year, JUL, 4)] = name
            if self.observed and date(year, JUL, 4).weekday() == SUN:
                self[date(year, JUL, 4) + rd(days=+1)] = name + " (Observed)"

        # Labor Day
        if year >= 1894:
            self[date(year, SEP, 1) + rd(weekday=MO)] = "Labor Day"

        # Veterans Day
        if year > 1953:
            name = "Veterans Day"
        else:
            name = "Armistice Day"
        if 1978 > year > 1970:
            self[date(year, OCT, 1) + rd(weekday=MO(+4))] = name
        elif year >= 1938:
            self[date(year, NOV, 11)] = name
            if self.observed \
                    and date(year, NOV, 11).weekday() == SUN:
                self[date(year, NOV, 11) + rd(days=+1)] = name + \
                    " (Observed)"

        # Thanksgiving
        if year > 1870:
            self[date(year, NOV, 1) + rd(weekday=TH(+4))] = "Thanksgiving"
            
        # Christmas Day
        if year > 1870:
            name = "Christmas Day"
            self[date(year, DEC, 25)] = "Christmas Day"
            if self.observed \
                    and date(year, DEC, 25).weekday() == SUN:
                self[date(year, DEC, 25) + rd(days=+1)] = name + \
                    " (Observed)"
        
us_holidays = UsElectricHolidays()

In [4]:
def label_weekends(df, timestamp_name):
    # Make the index the date
    df.index = df[timestamp_name]

    # Creates column for day of the week: 0 = Monday to 6 = Sunday
    df['DayWeek'] = df[timestamp_name].dt.dayofweek 
    # .apply run a function through every line of code without usong a for loop
    # lambda is a temporay function 
    # If x >= 5 set true (boolean statement) since it is the weekend # 5 is Saturday and 6 is Sunday
    # Returns boolean staement where weekend is true 
    df['Is_Weekend'] = df['DayWeek'].apply(lambda x: True if x >= 5 else False )
    # Creates an array for all the US Holidays in California
    # us_holidays = holidays.CountryHoliday('US')
    # Uses apply and lambda again to check if there is a holiday 
    # X in holiday returns a boolean satement 
    # .apply runs this function thorugh the entire Date column 
    # Returns a new boolean column where holidays is true 
    df['Is_Holiday'] = df[timestamp_name].apply(lambda x: True if x in us_holidays else False )
    # Creates a new column of Boolean satements of where summer is true
    # 6 is June and 9 is September
    df['is_summer'] = df[timestamp_name].dt.month.between(6,9)

    # This function determines the four catogories of Summer Weekend, Winter Weekend, Summer Weekday, Winter Weekday
    # The function takes in 4 columns in the dataframme
    # THhe .apply() function passes in one variable at a time from each column
    def peak_fun(vec):
        Date = vec[0] # First Column Date, .apply() Passes in a timestamp not a vector
        Is_Weekend = vec[1] # Second Column Is Weekend, .apply() Passes in a bool not a vector
        Is_Holiday = vec[2] # Third Column Is Holiday, .apply() Passes in a bool not a vector
        Is_Summer = vec[3] # Fourth Column Is Summer, .apply() Passes in a bool not a vector
        if Is_Weekend or Is_Holiday:
            if Is_Summer:
                return 'Summer_Weekend'       
            else:
                return 'Winter_Weekend'
        else:
            if Is_Summer:
                return 'Summer_Weekday'
            else:
                return 'Winter_Weekday'

    # Pass in the 4 columns and use the .apply() function
    df['fun'] = df[[timestamp_name, 'Is_Weekend', 'Is_Holiday', 'is_summer']].apply(peak_fun, axis = 1)
    return df 

In [5]:
# Edison TOU Pricing New

# Summer Weekday Time
su_wdy_sofpk = False
su_wdy_ofpk = ['0:00','16:00','21:00','0:00']
su_wdy_mdpk = False
su_wdy_onpk = ['16:00', '21:00']

# Summer Weekend Time
su_wkd_sofpk = False
su_wkd_ofpk = ['0:00','16:00','21:00','0:00']
su_wkd_mdpk = ['16:00', '21:00']
su_wkd_onpk = False

# Winter Weekday Time
wt_wdy_sofpk = ['8:00','16:00']
wt_wdy_ofpk = ['0:00','8:00','21:00','0:00']
wt_wdy_mdpk = ['16:00', '21:00']
wt_wdy_onpk = False

# Winter Weekend Time
wt_wkd_sofpk = ['8:00','16:00']
wt_wkd_ofpk = ['0:00','8:00','21:00','0:00']
wt_wkd_mdpk = ['16:00', '21:00']
wt_wkd_onpk = False

In [6]:
def summer_weekday(su_wdy_sofpk, su_wdy_ofpk, su_wdy_mdpk, su_wdy_onpk, df):
    
    if su_wdy_sofpk != False:
        len_su_wdy_sofpk = int(len(su_wdy_sofpk)/2)
        begin = 0
        end = 1
        for i in range(0,len_su_wdy_sofpk,1):
            selection = df[df['fun'] == 'Summer_Weekday'].between_time(su_wdy_sofpk[begin],su_wdy_sofpk[end], include_end=False)
            selection["Peak"] = "Super Off-Peak"
            ww = df["fun"] == "Summer_Weekday"
            df.loc[df[ww].between_time(su_wdy_sofpk[begin], su_wdy_sofpk[end], include_end=False).index, "Peak"] = "Super Off-Peak"
            begin += 2
            end += 2
            
    if su_wdy_ofpk != False:
        len_su_wdy_ofpk = int(len(su_wdy_ofpk)/2)
        begin = 0
        end = 1
        for i in range(0,len_su_wdy_ofpk,1):
            selection = df[df['fun'] == 'Summer_Weekday'].between_time(su_wdy_ofpk[begin],su_wdy_ofpk[end], include_end=False)
            selection["Peak"] = "Off-Peak"
            ww = df["fun"] == "Summer_Weekday"
            df.loc[df[ww].between_time(su_wdy_ofpk[begin], su_wdy_ofpk[end], include_end=False).index, "Peak"] = "Off-Peak"
            begin += 2
            end += 2
    
    if su_wdy_mdpk != False:
        len_su_wdy_mdpk = int(len(su_wdy_mdpk)/2)
        begin = 0
        end = 1
        for i in range(0,len_su_wdy_mdpk,1):
            selection = df[df['fun'] == 'Summer_Weekday'].between_time(su_wdy_mdpk[begin],su_wdy_mdpk[end], include_end=False)
            selection["Peak"] = "Mid-Peak"
            ww = df["fun"] == "Summer_Weekday"
            df.loc[df[ww].between_time(su_wdy_mdpk[begin], su_wdy_mdpk[end], include_end=False).index, "Peak"] = "Mid-Peak"
            begin += 2
            end += 2
    
    if su_wdy_onpk != False:
        len_su_wdy_onpk = int(len(su_wdy_onpk)/2)
        begin = 0
        end = 1
        for i in range(0,len_su_wdy_onpk,1):
            selection = df[df['fun'] == 'Summer_Weekday'].between_time(su_wdy_onpk[begin],su_wdy_onpk[end], include_end=False)
            selection["Peak"] = "On-Peak"
            ww = df["fun"] == "Summer_Weekday"
            df.loc[df[ww].between_time(su_wdy_onpk[begin], su_wdy_onpk[end], include_end=False).index, "Peak"] = "On-Peak"
            begin += 2
            end += 2
    return df

def summer_weekend(su_wkd_sofpk, su_wkd_ofpk, su_wkd_mdpk, su_wkd_onpk, df):
    
    if su_wkd_sofpk != False:
        len_su_wkd_sofpk = int(len(su_wkd_sofpk)/2)
        begin = 0
        end = 1
        for i in range(0,len_su_wkd_sofpk,1):
            selection = df[df['fun'] == 'Summer_Weekend'].between_time(su_wkd_sofpk[begin],su_wkd_sofpk[end], include_end=False)
            selection["Peak"] = "Super Off-Peak"
            ww = df["fun"] == "Summer_Weekend"
            df.loc[df[ww].between_time(su_wkd_sofpk[begin], su_wkd_sofpk[end], include_end=False).index, "Peak"] = "Super Off-Peak"
            begin += 2
            end += 2
            
    if su_wkd_ofpk != False:
        len_su_wkd_ofpk = int(len(su_wkd_ofpk)/2)
        begin = 0
        end = 1
        for i in range(0,len_su_wkd_ofpk,1):
            selection = df[df['fun'] == 'Summer_Weekend'].between_time(su_wkd_ofpk[begin],su_wkd_ofpk[end], include_end=False)
            selection["Peak"] = "Off-Peak"
            ww = df["fun"] == "Summer_Weekend"
            df.loc[df[ww].between_time(su_wkd_ofpk[begin], su_wkd_ofpk[end], include_end=False).index, "Peak"] = "Off-Peak"
            begin += 2
            end += 2
    
    if su_wkd_mdpk != False:
        len_su_wkd_mdpk = int(len(su_wkd_mdpk)/2)
        begin = 0
        end = 1
        for i in range(0,len_su_wkd_mdpk,1):
            selection = df[df['fun'] == 'Summer_Weekend'].between_time(su_wkd_mdpk[begin],su_wkd_mdpk[end], include_end=False)
            selection["Peak"] = "Mid-Peak"
            ww = df["fun"] == "Summer_Weekend"
            df.loc[df[ww].between_time(su_wkd_mdpk[begin], su_wkd_mdpk[end], include_end=False).index, "Peak"] = "Mid-Peak"
            begin += 2
            end += 2
    
    if su_wkd_onpk != False:
        len_su_wkd_onpk = int(len(su_wkd_onpk)/2)
        begin = 0
        end = 1
        for i in range(0,len_su_wkd_onpk,1):
            selection = df[df['fun'] == 'Summer_Weekend'].between_time(su_wkd_onpk[begin],su_wkd_onpk[end], include_end=False)
            selection["Peak"] = "On-Peak"
            ww = df["fun"] == "Summer_Weekend"
            df.loc[df[ww].between_time(su_wkd_onpk[begin], su_wkd_onpk[end], include_end=False).index, "Peak"] = "On-Peak"
            begin += 2
            end += 2
    return df

def winter_weekday(wt_wdy_sofpk, wt_wdy_ofpk, wt_wdy_mdpk, wt_wdy_onpk, df):
    
    if wt_wdy_sofpk != False:
        len_wt_wdy_sofpk = int(len(wt_wdy_sofpk)/2)
        begin = 0
        end = 1
        for i in range(0,len_wt_wdy_sofpk,1):
            selection = df[df['fun'] == 'Winter_Weekday'].between_time(wt_wdy_sofpk[begin],wt_wdy_sofpk[end], include_end=False)
            selection["Peak"] = "Super Off-Peak"
            ww = df["fun"] == "Winter_Weekday"
            df.loc[df[ww].between_time(wt_wdy_sofpk[begin], wt_wdy_sofpk[end], include_end=False).index, "Peak"] = "Super Off-Peak"
            begin += 2
            end += 2
            
    if wt_wdy_ofpk != False:
        len_wt_wdy_ofpk = int(len(wt_wdy_ofpk)/2)
        begin = 0
        end = 1
        for i in range(0,len_wt_wdy_ofpk,1):
            selection = df[df['fun'] == 'Winter_Weekday'].between_time(wt_wdy_ofpk[begin],wt_wdy_ofpk[end], include_end=False)
            selection["Peak"] = "Off-Peak"
            ww = df["fun"] == "Winter_Weekday"
            df.loc[df[ww].between_time(wt_wdy_ofpk[begin], wt_wdy_ofpk[end], include_end=False).index, "Peak"] = "Off-Peak"
            begin += 2
            end += 2
    
    if wt_wdy_mdpk != False:
        len_wt_wdy_mdpk = int(len(wt_wdy_mdpk)/2)
        begin = 0
        end = 1
        for i in range(0,len_wt_wdy_mdpk,1):
            selection = df[df['fun'] == 'Winter_Weekday'].between_time(wt_wdy_mdpk[begin],wt_wdy_mdpk[end], include_end=False)
            selection["Peak"] = "Mid-Peak"
            ww = df["fun"] == "Winter_Weekday"
            df.loc[df[ww].between_time(wt_wdy_mdpk[begin], wt_wdy_mdpk[end], include_end=False).index, "Peak"] = "Mid-Peak"
            begin += 2
            end += 2
    
    if wt_wdy_onpk != False:
        len_wt_wdy_onpk = int(len(wt_wdy_onpk)/2)
        begin = 0
        end = 1
        for i in range(0,len_wt_wdy_onpk,1):
            selection = df[df['fun'] == 'Winter_Weekday'].between_time(wt_wdy_onpk[begin],wt_wdy_onpk[end], include_end=False)
            selection["Peak"] = "On-Peak"
            ww = df["fun"] == "Winter_Weekday"
            df.loc[df[ww].between_time(wt_wdy_onpk[begin], wt_wdy_onpk[end], include_end=False).index, "Peak"] = "On-Peak"
            begin += 2
            end += 2
    return df

def winter_weekend(wt_wkd_sofpk, wt_wkd_ofpk, wt_wkd_mdpk, wt_wkd_onpk, df):
    
    if wt_wkd_sofpk != False:
        len_wt_wkd_sofpk = int(len(wt_wkd_sofpk)/2)
        begin = 0
        end = 1
        for i in range(0,len_wt_wkd_sofpk,1):
            selection = df[df['fun'] == 'Winter_Weekend'].between_time(wt_wkd_sofpk[begin],wt_wkd_sofpk[end], include_end=False)
            selection["Peak"] = "Super Off-Peak"
            ww = df["fun"] == "Winter_Weekend"
            df.loc[df[ww].between_time(wt_wkd_sofpk[begin], wt_wkd_sofpk[end], include_end=False).index, "Peak"] = "Super Off-Peak"
            begin += 2
            end += 2
            
    if wt_wkd_ofpk != False:
        len_wt_wkd_ofpk = int(len(wt_wkd_ofpk)/2)
        begin = 0
        end = 1
        for i in range(0,len_wt_wkd_ofpk,1):
            selection = df[df['fun'] == 'Winter_Weekend'].between_time(wt_wkd_ofpk[begin],wt_wkd_ofpk[end], include_end=False)
            selection["Peak"] = "Off-Peak"
            ww = df["fun"] == "Winter_Weekend"
            df.loc[df[ww].between_time(wt_wkd_ofpk[begin], wt_wkd_ofpk[end], include_end=False).index, "Peak"] = "Off-Peak"
            begin += 2
            end += 2
    
    if wt_wkd_mdpk != False:
        len_wt_wkd_mdpk = int(len(wt_wkd_mdpk)/2)
        begin = 0
        end = 1
        for i in range(0,len_wt_wkd_mdpk,1):
            selection = df[df['fun'] == 'Winter_Weekend'].between_time(wt_wkd_mdpk[begin],wt_wkd_mdpk[end], include_end=False)
            selection["Peak"] = "Mid-Peak"
            ww = df["fun"] == "Winter_Weekend"
            df.loc[df[ww].between_time(wt_wkd_mdpk[begin], wt_wkd_mdpk[end], include_end=False).index, "Peak"] = "Mid-Peak"
            begin += 2
            end += 2
    
    if wt_wkd_onpk != False:
        len_wt_wkd_onpk = int(len(wt_wkd_onpk)/2)
        begin = 0
        end = 1
        for i in range(0,len_wt_wkd_onpk,1):
            selection = df[df['fun'] == 'Winter_Weekend'].between_time(wt_wkd_onpk[begin],wt_wkd_onpk[end], include_end=False)
            selection["Peak"] = "On-Peak"
            ww = df["fun"] == "Winter_Weekend"
            df.loc[df[ww].between_time(wt_wkd_onpk[begin], wt_wkd_onpk[end], include_end=False).index, "Peak"] = "On-Peak"
            begin += 2
            end += 2
    return df

In [7]:
def label_peaks(df):
    winter_weekend(wt_wkd_sofpk, wt_wkd_ofpk, wt_wkd_mdpk, wt_wkd_onpk, df)
    winter_weekday(wt_wdy_sofpk, wt_wdy_ofpk, wt_wdy_mdpk, wt_wdy_onpk, df)
    summer_weekday(su_wdy_sofpk, su_wdy_ofpk, su_wdy_mdpk, su_wdy_onpk, df)
    summer_weekend(su_wkd_sofpk, su_wkd_ofpk, su_wkd_mdpk, su_wkd_onpk, df)

In [8]:
df = pd.read_csv('/home/sigi_laptop/Documents/Research/NAPS_2023/Code/Validation/microgrid_data.csv')
df

,time,building_load.P,generator.P,grid.sou.S[1]
0,0.000000e+00,-18887.000000,-93.0,-18980.000000
1,9.000000e+02,-21254.000000,-93.0,-21347.000000
2,9.000000e+02,-21254.000000,-93.0,-21347.000000
3,9.000000e+02,-21254.000000,-93.0,-21347.000000
4,1.800000e+03,-20770.000000,-93.0,-20863.000000
...,...,...,...,...
105115,3.153510e+07,-20537.000000,-93.0,-20630.000000
105116,3.153510e+07,-20537.000000,-93.0,-20630.000000
105117,3.153510e+07,-20536.999509,-93.0,-20629.999509
105118,3.153600e+07,-20397.000000,-93.0,-20490.000000


In [9]:
df['time'] = pd.to_datetime(df['time'] + 1646092800, unit = 's')
df

,time,building_load.P,generator.P,grid.sou.S[1]
0,2022-03-01 00:00:00.000000000,-18887.000000,-93.0,-18980.000000
1,2022-03-01 00:15:00.000000000,-21254.000000,-93.0,-21347.000000
2,2022-03-01 00:15:00.000000000,-21254.000000,-93.0,-21347.000000
3,2022-03-01 00:15:00.000000000,-21254.000000,-93.0,-21347.000000
4,2022-03-01 00:30:00.000000000,-20770.000000,-93.0,-20863.000000
...,...,...,...,...
105115,2023-02-28 23:45:00.000000000,-20537.000000,-93.0,-20630.000000
105116,2023-02-28 23:45:00.003153664,-20537.000000,-93.0,-20630.000000
105117,2023-02-28 23:45:00.003153664,-20536.999509,-93.0,-20629.999509
105118,2023-03-01 00:00:00.000000000,-20397.000000,-93.0,-20490.000000


In [10]:
df[df.select_dtypes(include=['number']).columns] /= 1000
df

,time,building_load.P,generator.P,grid.sou.S[1]
0,2022-03-01 00:00:00.000000000,-18.887,-0.093,-18.980
1,2022-03-01 00:15:00.000000000,-21.254,-0.093,-21.347
2,2022-03-01 00:15:00.000000000,-21.254,-0.093,-21.347
3,2022-03-01 00:15:00.000000000,-21.254,-0.093,-21.347
4,2022-03-01 00:30:00.000000000,-20.770,-0.093,-20.863
...,...,...,...,...
105115,2023-02-28 23:45:00.000000000,-20.537,-0.093,-20.630
105116,2023-02-28 23:45:00.003153664,-20.537,-0.093,-20.630
105117,2023-02-28 23:45:00.003153664,-20.537,-0.093,-20.630
105118,2023-03-01 00:00:00.000000000,-20.397,-0.093,-20.490


In [11]:
def first_resample(df, time_column = 'time', resample_rate = '15T'):
    df[time_column] = pd.to_datetime(df[time_column])
    df = df.set_index(df[time_column])
    df = df.drop(columns = [time_column])
    df = df.resample(resample_rate).mean()  #df.groupby(pd.Grouper(key=time_column, freq=resample_rate)).ffill().bfill() 
    df.insert(loc=0, column=time_column, value=df.index) #df.insert(loc=0, column=time_column, value=np.arange(0, len(df.index), 1, dtype=int)) #
    df = df.reset_index(drop=True)
    #df.fillna(0)
    #df['value'] = np.floor(pd.to_numeric(df['value'], errors='coerce')).astype('Int64')
    #df = df.fillna(0)
    return df

In [12]:
df = first_resample(df)
df

,time,building_load.P,generator.P,grid.sou.S[1]
0,2022-03-01 00:00:00,-18.887,-0.093,-18.980
1,2022-03-01 00:15:00,-21.254,-0.093,-21.347
2,2022-03-01 00:30:00,-20.770,-0.093,-20.863
3,2022-03-01 00:45:00,-21.144,-0.106,-21.250
4,2022-03-01 01:00:00,-20.331,-0.079,-20.410
...,...,...,...,...
35036,2023-02-28 23:00:00,-20.278,-0.093,-20.371
35037,2023-02-28 23:15:00,-20.578,-0.093,-20.671
35038,2023-02-28 23:30:00,-20.677,-0.093,-20.770
35039,2023-02-28 23:45:00,-20.537,-0.093,-20.630


In [13]:
act = pd.read_csv('/home/sigi_laptop/Documents/Research/NAPS_2023/Code/Validation/optimization_1084_2022_2023.csv')
act

,time,solar,meter_load,building_load,opt_load,opt_load_intplt,inverter_control
0,2022-03-01 00:00:00,-93.0,-18980.651823,-18887.651823,8078.5,8078.5,-27059.151823
1,2022-03-01 00:15:00,-93.0,-21347.288021,-21254.288021,8078.5,8078.5,-29425.788021
2,2022-03-01 00:30:00,-93.0,-20863.807031,-20770.807031,8078.5,8078.5,-28942.307031
3,2022-03-01 00:45:00,-106.0,-21250.331901,-21144.331901,8078.5,8078.5,-29328.831901
4,2022-03-01 01:00:00,-79.0,-20410.336589,-20331.336589,8078.5,8078.5,-28488.836589
...,...,...,...,...,...,...,...
35035,2023-02-28 22:45:00,-93.0,-20517.146615,-20424.146615,-6592.0,-6592.0,-13925.146615
35036,2023-02-28 23:00:00,-93.0,-20371.230339,-20278.230339,-6592.0,-6592.0,-13779.230339
35037,2023-02-28 23:15:00,-93.0,-20671.189193,-20578.189193,-6592.0,-6592.0,-14079.189193
35038,2023-02-28 23:30:00,-93.0,-20770.800651,-20677.800651,-6592.0,-6592.0,-14178.800651


In [15]:
act[act.select_dtypes(include=['number']).columns] /= 1000
act

,time,solar,meter_load,building_load,opt_load,opt_load_intplt,inverter_control
0,2022-03-01 00:00:00,-0.093,-18.980652,-18.887652,8.0785,8.0785,-27.059152
1,2022-03-01 00:15:00,-0.093,-21.347288,-21.254288,8.0785,8.0785,-29.425788
2,2022-03-01 00:30:00,-0.093,-20.863807,-20.770807,8.0785,8.0785,-28.942307
3,2022-03-01 00:45:00,-0.106,-21.250332,-21.144332,8.0785,8.0785,-29.328832
4,2022-03-01 01:00:00,-0.079,-20.410337,-20.331337,8.0785,8.0785,-28.488837
...,...,...,...,...,...,...,...
35035,2023-02-28 22:45:00,-0.093,-20.517147,-20.424147,-6.5920,-6.5920,-13.925147
35036,2023-02-28 23:00:00,-0.093,-20.371230,-20.278230,-6.5920,-6.5920,-13.779230
35037,2023-02-28 23:15:00,-0.093,-20.671189,-20.578189,-6.5920,-6.5920,-14.079189
35038,2023-02-28 23:30:00,-0.093,-20.770801,-20.677801,-6.5920,-6.5920,-14.178801


In [16]:
df['meter_load'] = act['meter_load']
df

,time,building_load.P,generator.P,grid.sou.S[1],meter_load
0,2022-03-01 00:00:00,-18.887,-0.093,-18.980,-18.980652
1,2022-03-01 00:15:00,-21.254,-0.093,-21.347,-21.347288
2,2022-03-01 00:30:00,-20.770,-0.093,-20.863,-20.863807
3,2022-03-01 00:45:00,-21.144,-0.106,-21.250,-21.250332
4,2022-03-01 01:00:00,-20.331,-0.079,-20.410,-20.410337
...,...,...,...,...,...
35036,2023-02-28 23:00:00,-20.278,-0.093,-20.371,-20.371230
35037,2023-02-28 23:15:00,-20.578,-0.093,-20.671,-20.671189
35038,2023-02-28 23:30:00,-20.677,-0.093,-20.770,-20.770801
35039,2023-02-28 23:45:00,-20.537,-0.093,-20.630,-20.630571


In [17]:
df.corr()

,building_load.P,generator.P,grid.sou.S[1],meter_load
building_load.P,1.000000,-0.662178,-0.377211,-0.342080
generator.P,-0.662178,1.000000,0.943771,0.902983
grid.sou.S[1],-0.377211,0.943771,1.000000,0.965087
meter_load,-0.342080,0.902983,0.965087,1.000000


Call the the function to label peaks

In [30]:
#df = clean_data(df, "Date", ['ServiceAccount', 'Date', 'Meter Number', 'Unit of Measurement'])
df = label_weekends(df, 'time')
label_peaks(df)
df = df.reset_index(drop = True)
df

/tmp/ipykernel_117457/34933441.py:161: FutureWarning: `include_start` and `include_end` are deprecated in favour of `inclusive`.
  selection = df[df['fun'] == 'Winter_Weekend'].between_time(wt_wkd_sofpk[begin],wt_wkd_sofpk[end], include_end=False)
/tmp/ipykernel_117457/34933441.py:162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selection["Peak"] = "Super Off-Peak"
/tmp/ipykernel_117457/34933441.py:164: FutureWarning: `include_start` and `include_end` are deprecated in favour of `inclusive`.
  df.loc[df[ww].between_time(wt_wkd_sofpk[begin], wt_wkd_sofpk[end], include_end=False).index, "Peak"] = "Super Off-Peak"
/tmp/ipykernel_117457/34933441.py:173: FutureWarning: `include_start` and `include_end` are deprecated in favour of `inclusive`.
  sele

,time,building_load.P,generator.P,grid.sou.S[1],meter_load,DayWeek,Is_Weekend,Is_Holiday,is_summer,fun,Peak
0,2022-03-01 00:00:00,-18.887,-0.093,-18.980,-18.980652,1,False,False,False,Winter_Weekday,Off-Peak
1,2022-03-01 00:15:00,-21.254,-0.093,-21.347,-21.347288,1,False,False,False,Winter_Weekday,Off-Peak
2,2022-03-01 00:30:00,-20.770,-0.093,-20.863,-20.863807,1,False,False,False,Winter_Weekday,Off-Peak
3,2022-03-01 00:45:00,-21.144,-0.106,-21.250,-21.250332,1,False,False,False,Winter_Weekday,Off-Peak
4,2022-03-01 01:00:00,-20.331,-0.079,-20.410,-20.410337,1,False,False,False,Winter_Weekday,Off-Peak
...,...,...,...,...,...,...,...,...,...,...,...
35036,2023-02-28 23:00:00,-20.278,-0.093,-20.371,-20.371230,1,False,False,False,Winter_Weekday,Off-Peak
35037,2023-02-28 23:15:00,-20.578,-0.093,-20.671,-20.671189,1,False,False,False,Winter_Weekday,Off-Peak
35038,2023-02-28 23:30:00,-20.677,-0.093,-20.770,-20.770801,1,False,False,False,Winter_Weekday,Off-Peak
35039,2023-02-28 23:45:00,-20.537,-0.093,-20.630,-20.630571,1,False,False,False,Winter_Weekday,Off-Peak


# Find Enrgy and Power Values and Costs

## Seperate interval by billing month

In [36]:
billing = pd.read_excel("/home/sigi_laptop/Documents/Research/NAPS_2023/Code/Microgrid_Data/billing_month.xlsx")
billing

,billing_month
0,2022-03-01
1,2022-04-01
2,2022-05-01
3,2022-06-01
4,2022-07-01
5,2022-08-01
6,2022-09-01
7,2022-10-01
8,2022-11-01
9,2022-12-01


In [37]:
billMonth = billing.billing_month
offset = pd.DateOffset(minutes=15)
timestamp_name = 'time'
month = {}
monthLength = len(billMonth) - 1
for i in range (0,monthLength,1):
    end = billMonth[i + 1] - offset
    month[i] = df[df[timestamp_name].between(billMonth[i], end)]
month

{0:                     time  building_load.P  generator.P  grid.sou.S[1]  \
 0    2022-03-01 00:00:00          -18.887       -0.093        -18.980   
 1    2022-03-01 00:15:00          -21.254       -0.093        -21.347   
 2    2022-03-01 00:30:00          -20.770       -0.093        -20.863   
 3    2022-03-01 00:45:00          -21.144       -0.106        -21.250   
 4    2022-03-01 01:00:00          -20.331       -0.079        -20.410   
 ...                  ...              ...          ...            ...   
 2971 2022-03-31 22:45:00          -16.041       -0.093        -16.134   
 2972 2022-03-31 23:00:00          -15.961       -0.079        -16.040   
 2973 2022-03-31 23:15:00          -16.713       -0.093        -16.806   
 2974 2022-03-31 23:30:00          -18.115       -0.079        -18.194   
 2975 2022-03-31 23:45:00          -15.920       -0.093        -16.013   
 
       meter_load  DayWeek  Is_Weekend  Is_Holiday  is_summer             fun  \
 0     -18.980652        1

## Find energy values (kWh) and energy costs

In [ ]:
def energy(df, month, time_column = 'time', demand = 'Demand', cost = [False, False, False, False, False]): 
    # cost = [] vector must have five numeric elements/items, or be set to False if not 
    df = df.reset_index(drop=True)

    if len(month) != 0:
        monthLength = len(month) 
    else:
         monthLength = 0
            
    monthTitle = {}
    for i in range(0, monthLength, 1): # iterates through every billing month df
        monthTitle[i] = month[i][time_column].min().strftime('%b_%d_%Y') + ' to ' + month[i][time_column].max().strftime('%b_%d_%Y')

    columns = ['Billing Month', 'kiloWattHour Total', 'kiloWattHour Super Off-Peak', 'kiloWattHour Off-Peak', 'kiloWattHour Mid-Peak', 'kiloWattHour On-Peak']
    columns_cost = ['Total Energy Cost ($)', 'Super Off-Peak Energy Cost ($)', 'Off-Peak Energy Cost ($)', 'Mid-Peak Energy Cost ($)', 'On-Peak Energy Cost ($)']
    for j in range(5):
            if (type(cost[j]) == int or type(cost[j]) == float ):
                columns.append(columns_cost[j])
    
    col_len = int(len(columns))
    kiloWattHour = pd.DataFrame(np.zeros([monthLength, col_len])*np.nan, columns = columns)

    for i in range(0, monthLength, 1):
        month[i] = month[i].reset_index(drop=True)
        kiloWattHour['Billing Month'][i] = monthTitle[i]
        kiloWattHour['kiloWattHour Total'][i] = month[i][demand].sum() / 4
        kiloWattHour['kiloWattHour Super Off-Peak'][i] = month[i][demand][month[i].Peak == 'Super Off-Peak'].sum() / 4
        kiloWattHour['kiloWattHour Off-Peak'][i] = month[i][demand][month[i].Peak == 'Off-Peak'].sum() / 4
        kiloWattHour['kiloWattHour Mid-Peak'][i] = month[i][demand][month[i].Peak == 'Mid-Peak'].sum() / 4
        kiloWattHour['kiloWattHour On-Peak'][i] = month[i][demand][month[i].Peak == 'On-Peak'].sum() / 4
        
        for k in range(5):
            if (type(cost[k]) == int or type(cost[k]) == float ):
                kiloWattHour[columns_cost[k]][i] = kiloWattHour[columns[k + 1]][i]  *  cost[k]
                

    kiloWattHour.loc[len(kiloWattHour)] = np.nan
    kiloWattHour = kiloWattHour.shift()
    kiloWattHour['Billing Month'][0] = 'Entire Interval'
    kiloWattHour['kiloWattHour Total'][0] = df[demand].sum() / 4
    kiloWattHour['kiloWattHour Super Off-Peak'][0] = df[demand][df.Peak == 'Super Off-Peak'].sum() / 4
    kiloWattHour['kiloWattHour Off-Peak'][0] = df[demand][df.Peak == 'Off-Peak'].sum() / 4
    kiloWattHour['kiloWattHour Mid-Peak'][0] = df[demand][df.Peak == 'Mid-Peak'].sum() / 4
    kiloWattHour['kiloWattHour On-Peak'][0] = df[demand][df.Peak == 'On-Peak'].sum() / 4
    for l in range(0, 5, 1):
        
        if (type(cost[l]) == int or type(cost[l]) == float ):
            kiloWattHour[columns_cost[l]][0] = kiloWattHour[columns[l + 1]][0] *  cost[l]
                

    return kiloWattHour 

In [ ]:
kiloWattHour_load = energy(df,month,time_column = 'time', demand = 'grid.sou.S[1]', cost = [0.1242, False, False, False, False])
kiloWattHour_load

In [ ]:
#kiloWattHour_load['Energy Cost ($)'] =  kiloWattHour_load['kiloWattHour Total'] * 0.1242
kiloWattHour_load[['Billing Month', 'kiloWattHour Total', 'Energy Cost ($)']].to_latex('/home/lencon/Git_Research/SIGI_Research_Meetings/05_apr_2023/Table/unopt_energy_costs_modelica.tex')

In [ ]:
kiloWattHour_opt_load = energy(df,month, demand = 'opt_load')
kiloWattHour_opt_load

In [ ]:
kiloWattHour_opt_load_intplt = energy(df,month, demand = 'opt_load_intplt')
kiloWattHour_opt_load_intplt

In [ ]:
kiloWattHour_opt_load_intplt['Energy Cost ($)'] =  kiloWattHour_opt_load_intplt['kiloWattHour Total'] * 0.1242
kiloWattHour_opt_load_intplt[['Billing Month', 'kiloWattHour Total', 'Energy Cost ($)']].to_latex('/home/lencon/Git_Research/SIGI_Research_Meetings/05_apr_2023/Table/opt_energy_costs.tex')

## Find power values (kW) and demand costs

In [ ]:
def max_peak(df, month, time_column = 'time', demand = 'Demand', cost = [False, False, False, False, False]):
    # cost = [] vector must have five numeric elements/items, or be set to False if not 
    df = df.reset_index(drop=True)

    if len(month) != 0:
        monthLength = len(month) 
    else:
         monthLength = 0

    monthTitle = {}
    for i in range(0, monthLength, 1): # iterates through every billing month df
        monthTitle[i] = month[i][time_column].min().strftime('%b_%d_%Y') + ' to ' + month[i][time_column].max().strftime('%b_%d_%Y')

    columns = ['Billing Month', 'max_peak Total', 'max_peak Super Off-Peak', 'max_peak Off-Peak', 'max_peak Mid-Peak', 'max_peak On-Peak']
    columns_cost = ['Total Demand Cost ($)', 'Super Off-Peak Demand Cost ($)', 'Off-Peak Demand Cost ($)', 'Mid-Peak Demand Cost ($)', 'On-Peak Demand Cost ($)']
    for j in range(5):
            if (type(cost[j]) == int or type(cost[j]) == float ):
                columns.append(columns_cost[j])
    col_len = int(len(columns))
    max_peak = pd.DataFrame(np.zeros([monthLength, col_len])*np.nan, columns = columns)
    for i in range(0, monthLength, 1):
        month[i] = month[i].reset_index(drop=True)
        max_peak['Billing Month'][i] = monthTitle[i]
        max_peak['max_peak Total'][i] = month[i][demand].max()
        max_peak['max_peak Super Off-Peak'][i] = month[i][demand][month[i].Peak == 'Super Off-Peak'].max()
        max_peak['max_peak Off-Peak'][i] = month[i][demand][month[i].Peak == 'Off-Peak'].max()
        max_peak['max_peak Mid-Peak'][i] = month[i][demand][month[i].Peak == 'Mid-Peak'].max()
        max_peak['max_peak On-Peak'][i] = month[i][demand][month[i].Peak == 'On-Peak'].max()
        for k in range(5):
            if (type(cost[k]) == int or type(cost[k]) == float ):
                max_peak[columns_cost[k]][i] = max_peak[columns[k + 1]][i]  *  cost[k]
                
    max_peak.loc[len(max_peak)] = np.nan
    max_peak = max_peak.shift()
    max_peak['Billing Month'][0] = 'Entire Interval'
    max_peak['max_peak Total'][0] = df[demand].max()
    max_peak['max_peak Super Off-Peak'][0] = df[demand][df.Peak == 'Super Off-Peak'].max()
    max_peak['max_peak Off-Peak'][0] = df[demand][df.Peak == 'Off-Peak'].max()
    max_peak['max_peak Mid-Peak'][0] = df[demand][df.Peak == 'Mid-Peak'].max()
    max_peak['max_peak On-Peak'][0] = df[demand][df.Peak == 'On-Peak'].max()
    
    for l in range(0, 5, 1):    
        if (type(cost[l]) == int or type(cost[l]) == float ):
            max_peak[columns_cost[l]][0] = max_peak[columns[l + 1]][0] *  cost[l]

    return max_peak

In [ ]:
max_peak_load = max_peak(df,month,time_column = 'time', demand = 'grid.sou.S[1]', cost = [12.48, False, False, False, False])
max_peak_load

In [ ]:
max_peak_load['Demand Cost ($)'] =  max_peak_load['max_peak Total'] * 12.48
max_peak_load[['Billing Month', 'max_peak Total', 'Demand Cost ($)']].to_latex('/home/lencon/Git_Research/SIGI_Research_Meetings/05_apr_2023/Table/unopt_demand_costs_modelica.tex')

In [ ]:
max_peak_opt_load = max_peak(df, month, demand = 'opt_load')
max_peak_opt_load

In [ ]:
max_peak_opt_load_intplt = max_peak(df, month, demand = 'opt_load_intplt')
max_peak_opt_load_intplt

In [ ]:
max_peak_opt_load_intplt['Demand Cost ($)'] =  max_peak_opt_load_intplt['max_peak Total'] * 12.48
max_peak_opt_load_intplt[['Billing Month', 'max_peak Total', 'Demand Cost ($)']].to_latex('/home/lencon/Git_Research/SIGI_Research_Meetings/05_apr_2023/Table/opt_demand_costs.tex')

In [ ]:
max_peak_load['Demand Cost ($)'][1:].sum() - max_peak_opt_load_intplt['Demand Cost ($)'][1:].sum()

In [ ]:
def energy_percentage(df, month):
    df = df.reset_index(drop=True)

    if len(month) != 0:
        monthLength = len(month) 
    else:
         monthLength = 0
         
    monthTitle = {}
    for i in range(0, monthLength, 1): # iterates through every billing month df
        monthTitle[i] = month[i].Date.min().strftime('%b_%d_%Y') + ' to ' + month[i].Date.max().strftime('%b_%d_%Y')

    columns = ['Billing Month', 'Energy_Percentage Total', 'Energy_Percentage Super Off-Peak', 'Energy_Percentage Off-Peak', 'Energy_Percentage Mid-Peak', 'Energy_Percentage On-Peak']
    col_len = int(len(columns))
    Energy_Percentage = pd.DataFrame(np.zeros([monthLength, col_len])*np.nan, columns = columns)
    for i in range(0, monthLength, 1):
        month[i] = month[i].reset_index(drop=True)
        Energy_Percentage['Billing Month'][i] = monthTitle[i]
        month_sum = month[i].Demand.sum()
        Energy_Percentage['Energy_Percentage Super Off-Peak'][i] = month[i].Demand[month[i].Peak == 'Super Off-Peak'].sum() / month_sum
        Energy_Percentage['Energy_Percentage Off-Peak'][i] = month[i].Demand[month[i].Peak == 'Off-Peak'].sum() / month_sum
        Energy_Percentage['Energy_Percentage Mid-Peak'][i] = month[i].Demand[month[i].Peak == 'Mid-Peak'].sum() / month_sum
        Energy_Percentage['Energy_Percentage On-Peak'][i] = month[i].Demand[month[i].Peak == 'On-Peak'].sum() / month_sum
        Energy_Percentage['Energy_Percentage Total'][i] = Energy_Percentage['Energy_Percentage Super Off-Peak'][i] + Energy_Percentage['Energy_Percentage Off-Peak'][i] + Energy_Percentage['Energy_Percentage Mid-Peak'][i] + Energy_Percentage['Energy_Percentage On-Peak'][i]
    

    Energy_Percentage.loc[len(Energy_Percentage)] = np.nan
    Energy_Percentage = Energy_Percentage.shift()
    Energy_Percentage['Billing Month'][0] = 'Entire Interval'
    df_sum = df.Demand.sum()
    Energy_Percentage['Energy_Percentage Super Off-Peak'][0] = df.Demand[df.Peak == 'Super Off-Peak'].sum() / df_sum
    Energy_Percentage['Energy_Percentage Off-Peak'][0] = df.Demand[df.Peak == 'Off-Peak'].sum() / df_sum
    Energy_Percentage['Energy_Percentage Mid-Peak'][0] = df.Demand[df.Peak == 'Mid-Peak'].sum() / df_sum
    Energy_Percentage['Energy_Percentage On-Peak'][0] = df.Demand[df.Peak == 'On-Peak'].sum() / df_sum
    Energy_Percentage['Energy_Percentage Total'][0] = Energy_Percentage['Energy_Percentage Super Off-Peak'][0] + Energy_Percentage['Energy_Percentage Off-Peak'][0] + Energy_Percentage['Energy_Percentage Mid-Peak'][0] + Energy_Percentage['Energy_Percentage On-Peak'][0]

    return Energy_Percentage

In [ ]:
kW_percentage = energy_percentage(df, month)
kW_percentage

## Find load factor

In [ ]:
def get_load_factor(df, month):    
    monthTitle = {}
    for j in range(0, monthLength, 1): # iterates through every billing month df
        monthTitle[j] = month[j].Date.min().strftime('%B %d, %Y') + ' to ' + month[j].Date.max().strftime('%B %d, %Y')

    columns = ['Billing Month', 'Load Factor Total', 'Load Factor Super Off-Peak', 'Load Factor Off-Peak', 'Load Factor Mid-Peak', 'Load Factor On-Peak']
    col_len = int(len(columns))
    loadFactor = pd.DataFrame(np.zeros([monthLength, col_len])*np.nan, columns = columns)
    for i in range(0, monthLength, 1):
        loadFactor['Billing Month'][i] = monthTitle[i]
        loadFactor['Load Factor Total'][i] = month[i].Demand.mean() / month[i].Demand.max()
        loadFactor['Load Factor Super Off-Peak'][i] = month[i].Demand[month[i].Peak == 'Super Off-Peak'].mean() / month[i].Demand[month[i].Peak == 'Super Off-Peak'].max()
        loadFactor['Load Factor Off-Peak'][i] = month[i].Demand[month[i].Peak == 'Off-Peak'].mean() / month[i].Demand[month[i].Peak == 'Off-Peak'].max()
        loadFactor['Load Factor Mid-Peak'][i] = month[i].Demand[month[i].Peak == 'Mid-Peak'].mean() / month[i].Demand[month[i].Peak == 'Mid-Peak'].max()
        loadFactor['Load Factor On-Peak'][i] = month[i].Demand[month[i].Peak == 'On-Peak'].mean() / month[i].Demand[month[i].Peak == 'On-Peak'].max()

    loadFactor.loc[len(loadFactor)] = np.nan
    loadFactor = loadFactor.shift()
    loadFactor['Billing Month'][0] = 'Entire Interval'
    loadFactor['Load Factor Total'][0] = df.Demand.mean() / df.Demand.max()
    loadFactor['Load Factor Super Off-Peak'][0] = df.Demand[df.Peak == 'Super Off-Peak'].mean() / df.Demand[df.Peak == 'Super Off-Peak'].max()
    loadFactor['Load Factor Off-Peak'][0] = df.Demand[df.Peak == 'Off-Peak'].mean() / df.Demand[df.Peak == 'Off-Peak'].max()
    loadFactor['Load Factor Mid-Peak'][0] = df.Demand[df.Peak == 'Mid-Peak'].mean() / df.Demand[df.Peak == 'Mid-Peak'].max()
    loadFactor['Load Factor On-Peak'][0] = df.Demand[df.Peak == 'On-Peak'].mean() / df.Demand[df.Peak == 'On-Peak'].max()

    return loadFactor

In [ ]:
loadFactor = get_load_factor(df, month)
loadFactor

## Plot Data 

In [18]:
import os
os.chdir('/home/sigi_laptop/Documents/Research/NAPS_2023/Code/Validation')

In [1]:
def plot_graph(x_data, y_data, line_name, title, x_title, y_title, filename, line_color, fig_show = False):
    # Define the figure
    fig = go.Figure()
    # Add data to figure
    for i in range(0, len(x_data), 1):
        fig.add_trace(go.Scattergl(x=x_data[i], y=y_data[i], opacity = 0.5, name= line_name[i], line_color = line_color[i]))
            
    # Create Layout for the Html
    
    fig.update_layout(
        title=go.layout.Title(
            text= title,
            xref="paper",
            x=0.5
        ),
        xaxis=go.layout.XAxis(
            title=go.layout.xaxis.Title(
                text=x_title,
            ),
            rangeslider=dict(
                visible=True
            )
        ),
        yaxis=go.layout.YAxis(
            title=go.layout.yaxis.Title(
                text=y_title,
            )
        ),
        plot_bgcolor = "white"
    )
    
    fig.write_html((filename + '.html'), include_mathjax='cdn')
    fig = go.Figure()
    # Add data to figure
    for i in range(0, len(x_data), 1):
        fig.add_trace(go.Scatter(x=x_data[i], y=y_data[i], opacity = 0.5, name= line_name[i], line_color = line_color[i]))
    fig.update_layout(
        title=go.layout.Title(
            text= title,
            xref="paper",
            x=0.5
        ),
        xaxis=go.layout.XAxis(
            title=go.layout.xaxis.Title(
                text=x_title,
            ),
            rangeslider=dict(
                visible=False
            )
        ),
        yaxis=go.layout.YAxis(
            title=go.layout.yaxis.Title(
                text=y_title,
            )
        ),
        plot_bgcolor = "white"
    )
    fig.write_image(filename + '.svg', engine = 'orca')
    fig.write_image(filename + '.pdf', engine = 'orca')
    # fig.write_image(filename + '.png') Do not use png in your code this is only an example to compare png to vector images
    if fig_show:
        fig.show()

In [40]:
#create title for plots
#create filename for plots
site_name = "1084 Modelica Simulation with Bess"
plot_type = "15 Minute Data"
month_title = {}
for i in range (0,len(month)):
    month_title[i] = site_name + "<br>" + plot_type + "<br>From: " +  month[i].time.min().strftime('%B %d, %Y') + " to " + month[i].time.max().strftime('%B %d, %Y') 
month_title

{0: '1084 Modelica Simulation with Bess<br>15 Minute Data<br>From: March 01, 2022 to March 31, 2022',
 1: '1084 Modelica Simulation with Bess<br>15 Minute Data<br>From: April 01, 2022 to April 30, 2022',
 2: '1084 Modelica Simulation with Bess<br>15 Minute Data<br>From: May 01, 2022 to May 31, 2022',
 3: '1084 Modelica Simulation with Bess<br>15 Minute Data<br>From: June 01, 2022 to June 30, 2022',
 4: '1084 Modelica Simulation with Bess<br>15 Minute Data<br>From: July 01, 2022 to July 31, 2022',
 5: '1084 Modelica Simulation with Bess<br>15 Minute Data<br>From: August 01, 2022 to August 31, 2022',
 6: '1084 Modelica Simulation with Bess<br>15 Minute Data<br>From: September 01, 2022 to September 30, 2022',
 7: '1084 Modelica Simulation with Bess<br>15 Minute Data<br>From: October 01, 2022 to October 31, 2022',
 8: '1084 Modelica Simulation with Bess<br>15 Minute Data<br>From: November 01, 2022 to November 30, 2022',
 9: '1084 Modelica Simulation with Bess<br>15 Minute Data<br>From: Dec

In [41]:
#create filename for plots
site_name = "1084_unopt_opt_BESS_mo"
plot_type = "15_Minute_Data"
filename_list = {}
for i in range (0,len(month)):
    filename_list[i] = str(i) + '_' + site_name+ '_' + plot_type + '_' + month[i].time.min().strftime('%b_%d_%Y') + "_to_" + month[i].time.max().strftime('%b_%d_%Y') 
filename_list

{0: '0_1084_unopt_opt_BESS_mo_15_Minute_Data_Mar_01_2022_to_Mar_31_2022',
 1: '1_1084_unopt_opt_BESS_mo_15_Minute_Data_Apr_01_2022_to_Apr_30_2022',
 2: '2_1084_unopt_opt_BESS_mo_15_Minute_Data_May_01_2022_to_May_31_2022',
 3: '3_1084_unopt_opt_BESS_mo_15_Minute_Data_Jun_01_2022_to_Jun_30_2022',
 4: '4_1084_unopt_opt_BESS_mo_15_Minute_Data_Jul_01_2022_to_Jul_31_2022',
 5: '5_1084_unopt_opt_BESS_mo_15_Minute_Data_Aug_01_2022_to_Aug_31_2022',
 6: '6_1084_unopt_opt_BESS_mo_15_Minute_Data_Sep_01_2022_to_Sep_30_2022',
 7: '7_1084_unopt_opt_BESS_mo_15_Minute_Data_Oct_01_2022_to_Oct_31_2022',
 8: '8_1084_unopt_opt_BESS_mo_15_Minute_Data_Nov_01_2022_to_Nov_30_2022',
 9: '9_1084_unopt_opt_BESS_mo_15_Minute_Data_Dec_01_2022_to_Dec_31_2022',
 10: '10_1084_unopt_opt_BESS_mo_15_Minute_Data_Jan_01_2023_to_Jan_31_2023',
 11: '11_1084_unopt_opt_BESS_mo_15_Minute_Data_Feb_01_2023_to_Feb_28_2023'}

In [ ]:
x_title = "Time"
y_title = "Demand (kW)"
line_name = ["Building Load", "BESS", "Solar", "Grid Meter"]
line_color = ["deepskyblue", "red", "green", "orange"]
for i in range (0,len(month)):
    x_data = [month[i].time] * 4
    y_data = [month[i]["building_load.P"], month[i]["BESS.P"], month[i]["generator.P"], month[i]["grid.sou.S[1]"]]
    title = month_title[i]
    filename = filename_list[i]
    plot_graph(x_data, y_data, line_name, title, x_title, y_title, filename, line_color)

In [ ]:
x_title = "Time"
y_title = "Demand (kW)"
line_name = ["Building Load", "BESS", "Solar", "Grid Meter"]
line_color = ["deepskyblue", "red", "green", "orange"]
x_data = [df.time] * 4
y_data = [df["building_load.P"], df["BESS.P"], df["generator.P"], df["grid.sou.S[1]"]]
site_name_t = "1084 Modelica Simulation with Bess"
plot_type_t = "15 Minute Data"
site_name_f = "1084_unopt_opt_BESS_mo"
plot_type_f = "15_Minute_Data"
title = site_name_t + "<br>" + plot_type_t + "<br>From: " +  df.time.min().strftime('%B %d, %Y') + " to " + df.time.max().strftime('%B %d, %Y') 
filename = site_name_f + '_' + plot_type_f + '_' + df.time.min().strftime('%b_%d_%Y') + "_to_" + df.time.max().strftime('%b_%d_%Y') 
plot_graph(x_data, y_data, line_name, title, x_title, y_title, filename, line_color)

In [20]:
x_title = "Time"
y_title = "Demand (kW)"
line_name = ["Grid Meter (Real Data)", "Grid Meter (Modelica Simulation)"]
line_color = ["deepskyblue", "red"]
x_data = [act.time, df.time] 
y_data = [act.meter_load, df["grid.sou.S[1]"]]
site_name_t = "Modelica Validation"
plot_type_t = "15 Minute Data"
site_name_f = "ucr"
plot_type_f = "15_Minute_Data"
title = site_name_t + "<br>" + plot_type_t + "<br>From: " +  df.time.min().strftime('%B %d, %Y') + " to " + df.time.max().strftime('%B %d, %Y') 
filename = site_name_f + '_' + plot_type_f + '_' + df.time.min().strftime('%b_%d_%Y') + "_to_" + df.time.max().strftime('%b_%d_%Y') 
plot_graph(x_data, y_data, line_name, title, x_title, y_title, filename, line_color)

Fontconfig warning: "/usr/share/fontconfig/conf.avail/05-reset-dirs-sample.conf", line 6: unknown element "reset-dirs"
[29919:0812/202908.499690:ERROR:buffer_manager.cc(488)] [.DisplayCompositor]GL ERROR :GL_INVALID_OPERATION : glBufferData: <- error from previous GL command
[29919:0812/202918.648017:ERROR:buffer_manager.cc(488)] [.DisplayCompositor]GL ERROR :GL_INVALID_OPERATION : glBufferData: <- error from previous GL command


In [21]:
x_title = "Time"
y_title = "Demand (kW)"
line_name = ["Grid Meter (Real Data)", "Grid Meter (Modelica Simulation)"]
line_color = ["deepskyblue", "red"]
x_data = [act['time'][act['time'].between('06-01-2022', '07-01-2022')], df['time'][df['time'].between('06-01-2022', '07-01-2022')]] 
y_data = [act['meter_load'][act['time'].between('06-01-2022', '07-01-2022')], df["grid.sou.S[1]"][df['time'].between('06-01-2022', '07-01-2022')]]
site_name_t = "Modelica Validation"
plot_type_t = "15 Minute Data"
site_name_f = "ucr"
plot_type_f = "15_Minute_Data_Jun"
title = site_name_t + "<br>" + plot_type_t + "<br>From: " +  x_data[1].min().strftime('%B %d, %Y') + " to " + x_data[1].max().strftime('%B %d, %Y') 
filename = site_name_f + '_' + plot_type_f + '_' + x_data[1].min().strftime('%b_%d_%Y') + "_to_" + x_data[1].max().strftime('%b_%d_%Y') 
plot_graph(x_data, y_data, line_name, title, x_title, y_title, filename, line_color)

[29919:0812/202920.955283:ERROR:buffer_manager.cc(488)] [.DisplayCompositor]GL ERROR :GL_INVALID_OPERATION : glBufferData: <- error from previous GL command
